In [33]:
#Prerna Singh Cell-Detection Project
# Tradional 3D CNN - Training using Clarity Data

In [34]:
#Imports
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

import random

import numpy as np
import pdb

import matplotlib.pyplot as plt
import os
import pylab
from numpy import random

In [35]:
#Hyperparamters
num_epochs = 200
batch_size = 5
learning_rate = 1e-3
momentum = 0.9
dr = 0.3
weight_decay = 1e-4

In [61]:
# 0. Loading Data
#
# loading extracted cells and non cell Regions Of Interest

ROI6L = np.load('ROI6L_bias_corrected.npy') 
ROI12R = np.load('ROI12R_bias_corrected.npy')
ROI6R = np.load('ROI6L_bias_corrected.npy')
ROI11R = np.load('ROI11R_bias_corrected.npy')
ROI9R = np.load('ROI9R_bias_corrected.npy')
FN2=np.load('FN2.npy')
ROI_TRAIN = np.vstack((ROI6L,ROI12R,ROI6R,ROI11R,ROI9R,FN2))
print("Total Number of Cell Regions: ", len(ROI_TRAIN))

NCR6L = np.load('NCR6L_bias_corrected.npy') 
NCR12R = np.load('NCR12R_bias_corrected.npy')
NCR6R = np.load('NCR6L_bias_corrected.npy')
NCR11R = np.load('NCR11R_bias_corrected.npy')
NCR9R = np.load('NCR9R_bias_corrected.npy')
FP2=np.load('FP2.npy')
NCR_TRAIN = np.vstack((NCR6L,NCR12R,NCR6R,NCR11R,NCR9R,FP2))
print("Number of Non-Cell Regions:",len(NCR_TRAIN))

NCR_TRAIN = np.asarray(NCR_TRAIN)
print(NCR_TRAIN.shape)

ROI_TRAIN = np.asarray(ROI_TRAIN)
print(ROI_TRAIN.shape)

ROI12L = np.load('ROI12L_bias_corrected.npy') 
ROI9L = np.load('ROI9L_bias_corrected.npy') 
ROI10R = np.load('ROI10R_bias_corrected.npy')
ROI_TEST = np.vstack((ROI12L,ROI9L,ROI10R))
print("Total Number of Cell Regions: ", len(ROI_TEST))

NCR12L = np.load('NCR12L_bias_corrected.npy') 
NCR9L = np.load('NCR9L_bias_corrected.npy') 
NCR10R = np.load('NCR10R_bias_corrected.npy')
NCR_TEST = np.vstack((NCR12L,NCR9L,NCR10R))
print("Number of Non-Cell Regions:",len(NCR_TEST))

NCR_TEST = np.asarray(NCR_TEST)
print(NCR_TEST.shape)

ROI_TEST = np.asarray(ROI_TEST)
print(ROI_TEST.shape)

Total Number of Cell Regions:  2028
Number of Non-Cell Regions: 19682
(19682, 12, 12, 12)
(2028, 12, 12, 12)
Total Number of Cell Regions:  607
Number of Non-Cell Regions: 6070
(6070, 12, 12, 12)
(607, 12, 12, 12)


In [62]:
#training and testing dataset

train = np.vstack((ROI_TRAIN,NCR_TRAIN))
#Train Data: 0-1545 = cell
#Train Data: 1545-16995 = non cell region
print('Number of Items in Train Data: %s' % len(train))
train = np.expand_dims(train, axis=1)
# expanded to fit network architecture of 1 channel. 
print('Shape of Train Data Vector:', train.shape)

test = np.vstack((ROI_TEST, NCR_TEST))
#Test Data: 0-956 = cell
#Test Data: 956-10516 = non cell region
print("Number of Items in Test Data: %s" %len(test))
test = np.expand_dims(test, axis=1)
print('Shape of Test Data Vector:', test.shape)

Number of Items in Train Data: 21710
Shape of Train Data Vector: (21710, 1, 12, 12, 12)
Number of Items in Test Data: 6677
Shape of Test Data Vector: (6677, 1, 12, 12, 12)


In [63]:
#Creating labels for training and testing dataset

target_train = []
target_test =[]

for i in range(0,2028):
    target_train.append(np.array([1]))
# create 1085 labels for Yes Cell.

for i in range(2028,21710):
    target_train.append(np.array([0]))
# create 1085 labels for No Cell.

for i in range(0,607):
    target_test.append(np.array([1]))
# create 275 labels for Yes Cell.

for i in range(0,6070):
    target_test.append(np.array([0]))
# create 275 labels for No Cell.

target_train = np.array(target_train)
target_test = np.array(target_test)

In [64]:
#train dataset conversion: np array -> torch tensor -> torch dataset

train_dataset = data.TensorDataset(torch.from_numpy(train), torch.from_numpy(target_train))
# create a custom tensor train dataset from numpy array. 
#print(torch.from_numpy(target_train).size())
train_loader = data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
# batch dataloader

In [65]:
#test dataset conversion: np array -> torch tensor -> torch dataset
test_dataset = data.TensorDataset(torch.from_numpy(test), torch.from_numpy(target_test))
print(torch.from_numpy(test).size())
print(torch.from_numpy(target_test).size())
# create a custom tensor test dataset from numpy array.

test_loader = data.DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)
# batch dataloader.
# no shuffling for test dataset.

torch.Size([6677, 1, 12, 12, 12])
torch.Size([6677, 1])


In [66]:
# 1. Model Architecture
# INPUT: 
# - 1 x 12 x 12 x 12 image
# CONV1: 3d CONV
# MAXPOOL: 3d MP
# CONV2: 3d CONV
# CONV3: 3d CONV
# FC1: Fully Connected Layer
# FC2: Fully Connected Layer

class M1(nn.Module):
    def __init__(self):
        super(M1, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv3d(1,64, (5,5,5),padding = 2),nn.Dropout(dr)) # in_channel, out_channel, kernel
        self.pool = nn.MaxPool3d((2, 2, 2),2) # kernel, stride
        self.conv2 = nn.Sequential(nn.Conv3d(64, 64, (3,3,3), padding = 1), nn.Dropout(dr)) # in_channel, out_channel, kernel
        self.conv3 = nn.Sequential(nn.Conv3d(64, 64, (3,3,3), padding = 1), nn.Dropout(dr)) # in_channel, out_channel, kernel
        self.fc1 = nn.Sequential(nn.Linear(6*6*6*64,150),nn.Dropout(dr))
        self.fc2 = nn.Sequential(nn.Linear(150,1),nn.Dropout(dr))

    def forward(self,x):
#        print('Input Shape: ', x.shape)
        x = F.relu(self.conv1(x))
#        print('Shape after CONV1: ', x.shape)
        # Conv1 Activation: ReLU
        x = self.pool(x)
#         print('Shape after Maxpool: ', x.shape)
        # Followed by Maxpool
     
        x = F.relu(self.conv2(x))
#         print('Shape after CONV2: ', x.shape)
        # Conv2 Activation: ReLU
        
        x = F.relu(self.conv3(x))
#         print('Shape after CONV3: ', x.shape)
        # Conv3 Activation: ReLU
        
        x = x.reshape(x.size(0),-1)
#         print('Shape after flatten: ', x.shape)
        # flatten vector
              
        x = F.relu(self.fc1(x))
        # FC1 Activation: ReLU
#         print('Shape after FC1: ', x.shape)
        x = self.fc2(x)
        # FC2 Activation: None
#         print('Shape after FC2: ', x.shape)
        m = nn.Sigmoid()
#         print('Shape after Sigmoid Output: ', x.shape)

        x = m(x)
        # Output Activation: Sigmoid
        return x

m1 = M1()

In [67]:
# 2. Loss and Optimizer

criterion = nn.BCELoss()
# Binary Cross Entropy Loss

optimizer = optim.SGD(m1.parameters(), lr = learning_rate, weight_decay = weight_decay)
# SGD Optimizer
# Regularization: Weight Decay

In [68]:
# Function for Validation Accuracy (to be used during training as well)

def testAccuracy(test_loader, m1):
    m1.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    m1.to(device)
    with torch.no_grad(): # don't compute grads because we're not backpropagating.
        
        correct = 0 # number of correctly guessed cases
        valloss = 0 # validation loss
        total = 0 # total number of cases
        
        # Use Test Dataset to Compute Validation Loss
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            images = images.float()
            labels = labels.float()


            outputs = m1(images)
            predicted = (outputs>0.5).float()

            
            #print("target label: ",labels)
            # What the target label is
            
            #print("predicted label: ",predicted)
            # What the model predicts the label to be

            # go through the vector of predicted labels
            for i in range(len(predicted)):
                if predicted[i] == labels[i]:
                    correct += 1
                    
            #print('number of correct matches: ', correct)
            # number of correct label matches
            total += len(labels)
            
            loss = criterion(outputs, labels)
            # compute loss between target label and predicted output label (unrounded this time)
            
            valloss += loss.item()
            # should this be +=?

        
    
    accruacy = correct/total * 100
    return valloss, accruacy
    #print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / 38))

In [69]:
# 3. Training Loop

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
m1.to(device)

total_loss = []
total_accuracy = []
valloss =[]
total_step = len(train_loader)
for epoch in range(num_epochs):
    steploss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        #print("image",images.shape)
        labels = labels.to(device)
        #print(labels.shape)
        images = images.float()
        labels = labels.float()
        
        outputs = m1(images)
        #print("ouput shape: ",outputs.shape)
        #print("outputs: ", outputs)
        #pdb.set_trace()
        #print('target label:',labels)
        #loss = criterion(outputs, torch.max(labels, 1)[1])
        loss = criterion(outputs, labels)
        steploss += loss.item()
       
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
    total_loss.append(steploss)
    total_accuracy.append(testAccuracy(test_loader, m1)[1])
    
    valloss.append(testAccuracy(test_loader,m1)[0])

In [70]:
total_accuracy

[98.18780889621087,
 98.38250711397333,
 98.75692676351655,
 99.19125355698667,
 99.14632319904149,
 99.20623034296841,
 99.23618391493186,
 98.98157855324247,
 99.25116070091359,
 98.87674105137037,
 99.25116070091359,
 99.19125355698667,
 99.07143926913284,
 99.1612999850232,
 97.8882731765763,
 99.17627677100495,
 99.1612999850232,
 99.20623034296841,
 99.3110678448405,
 99.34102141680395,
 99.41590534671259,
 99.40092856073088,
 99.20623034296841,
 97.78343567470421,
 98.87674105137037,
 99.46083570465778,
 99.35599820278568,
 99.3110678448405,
 99.28111427287703,
 99.22120712895013,
 99.41590534671259,
 99.49078927662124,
 99.29609105885876,
 99.44585891867605,
 98.92167140931556,
 99.14632319904149,
 99.46083570465778,
 99.43088213269432,
 99.3110678448405,
 99.41590534671259,
 99.38595177474913,
 99.46083570465778,
 99.4758124906395,
 99.50576606260296,
 99.5207428485847,
 99.3110678448405,
 99.29609105885876,
 99.46083570465778,
 99.38595177474913,
 99.37097498876741,
 99.38595

In [71]:
torch.save(m1.state_dict(), 'coarsevalidation2retrain.pt')

In [59]:
#how to load model
#m1 = M1()
#m1.load_state_dict(torch.load('model_trained_on6L6R9L.pt'))